In [2]:
%matplotlib inline
import unifier

import sys
import matplotlib.pyplot as plt

from src.data.preprocess import run_pca
from src.data.make_dataset import load_dataset, load_subjects,make_mous_dataset, read_hcp

from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import hcp
import mne
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier as KNN


import os

In [5]:
sampleName = '107473'
HCP_DATA_FOLDER = os.getcwd() + '/HCP'
sample = read_hcp(sampleName,HCP_DATA_FOLDER,1)
PARTITIONS = 5

positions,polarPositions,partitions,channelData = unifier.spacialPartitionSensors(sample,PARTITIONS)

In [6]:
processedPath = os.getcwd() + "/Repo/age-classifier/data"

dataset = load_dataset(processedPath+"/processed/hcp",processedPath+"/hcp-eval-dist.csv")
df = pd.DataFrame.from_dict(dataset).set_index('id')
df['data'] = df['data'].map(lambda x : x[0])

SENSOR_COUNT = df['data'][0].shape[0]
FREQUENCY_COUNT = df['data'][0].shape[1]

test = np.array([[df['data'].map(lambda x: x[S,F]) for F in range(FREQUENCY_COUNT)] for S in range(SENSOR_COUNT)])

mean = test.mean(axis=2)
std = test.std(axis=2)

df['dataStandardized'] = df['data'].map(lambda x: (x-mean)/std)

channelKinds = np.array([e['kind']._name for e in channelData])
mask = np.array([(e == 'FIFFV_MEG_CH') for e in channelKinds])

positions, polarPositions, partitions = positions[mask],polarPositions[mask],partitions[mask]

CHANNELS = 24
BIN_SIZE = PARTITIONS + 1

def transformChannels(data,partitions):
    bins = [[[]]*BIN_SIZE]*BIN_SIZE
    for e,index in zip(data,partitions):
        bins[index[0].astype('int')][index[1].astype('int')].append(e)
        
    data = np.ndarray((BIN_SIZE,BIN_SIZE,CHANNELS))
    
    for i in range(BIN_SIZE):
        for j in range(BIN_SIZE):
            data[i][j] = np.mean(bins[i][j],axis=0) 
    
    return data

df['dataBinned'] = df['dataStandardized'].map(lambda x : transformChannels(x,partitions))

dfout = df[['age', 'gender','dataBinned']]
dfout['dataBinned'] = dfout['dataBinned'].map(lambda x: x.ravel())

dfout.to_csv("processed-eval.csv")

/Users/billymihalarias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
print("data points before processing {}".format(df['data'][0].shape[0] * df['data'][0].shape[1]))

data points before processing 5952


In [10]:
print("data points after processing {}".format(df['dataBinned'][0].shape[0] * df['dataBinned'][0].shape[1] * df['dataBinned'][0].shape[2]))

data points after processing 864


In [11]:
X = np.stack(df['dataBinned'].map(lambda x: x.flatten()).values)
y = df['age'].values

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [13]:
age_model = svm.SVC(gamma=0.01, C=10.)
age_model.fit(X_train, y_train)
None

In [14]:
print("{}%".format(round(age_model.score(X_test,y_test)*100),2))
print(confusion_matrix(y_test, age_model.predict(X_test),labels=['22-25', '26-30', '31-35']))

97.0%
[[201   3   7]
 [  5 204   0]
 [  0   2 205]]


In [15]:
age_model2 = KNN()
age_model2.fit(X_train, y_train)
None

In [16]:
print("{}%".format(round(age_model2.score(X_test,y_test)*100),2) )
print(confusion_matrix(y_test, age_model2.predict(X_test),labels=['22-25', '26-30', '31-35']))
print("\nThis accuracy was 99.5% pre grouping")

95.0%
[[198   6   7]
 [  5 203   1]
 [  3   7 197]]

This accuracy was 99.5% pre grouping


In [17]:
dfout = df[['age', 'gender','dataBinned']]
dfout['dataBinned'] = dfout['dataBinned'].map(lambda x: x.ravel())

dfout.to_csv("processed.csv")

/Users/billymihalarias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
dfout['dataBinned'][0]

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [19]:
df['data'][0].shape

(248, 24)